In [3]:
import folium
import ee
import geemap
import geopandas as gpd
ee.Authenticate()

# Write your project ID here, in quotes
ee.Initialize(project = "anr-41793")


Successfully saved authorization token.


In [4]:
start_date = '2025-01-01'
end_date = '2025-01-31'

In [5]:
map = folium.Map(location=[28.263363, -83.497652], tiles="Cartodb dark_matter", zoom_start=7)

In [6]:
def add_ee_layer(self, ee_image_object, vis_params, name):
    """Adds a method for displaying Earth Engine image tiles to folium map."""
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer

In [7]:
fl = gpd.read_file('https://github.com/geo-di-lab/emerge-lessons/raw/refs/heads/main/docs/data/florida_boundary.geojson')[['geometry']]
region = geemap.geopandas_to_ee(fl)

In [8]:
lst = (
    ee.ImageCollection('MODIS/061/MOD11A1')
      .filterDate(start_date, end_date)
      .select('LST_Day_1km')
      .median()
      .clip(region)
)

lst_vis = {
    'min': 13000.0,
    'max': 16500.0,
    'palette': [
    '040274', '040281', '0502a3', '0502b8', '0502ce', '0502e6',
    '0602ff', '235cb1', '307ef3', '269db1', '30c8e2', '32d3ef',
    '3be285', '3ff38f', '86e26f', '3ae237', 'b5e22e', 'd6e21f',
    'fff705', 'ffd611', 'ffb613', 'ff8b13', 'ff6e08', 'ff500d',
    'ff0000', 'de0101', 'c21301', 'a71001', '911003'
  ],
}

map.add_ee_layer(lst, lst_vis, "LST")

display(map)

In [9]:
def mask_s2_clouds(image):
  """Masks clouds in a Sentinel-2 image using the QA band.
  Args:
      image (ee.Image): A Sentinel-2 image.
  Returns:
      ee.Image: A cloud-masked Sentinel-2 image.
  """
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloud_bit_mask = 1 << 10
  cirrus_bit_mask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = (
      qa.bitwiseAnd(cloud_bit_mask)
      .eq(0)
      .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
  )

  return image.updateMask(mask).divide(10000)

In [10]:
rgb = (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
      .filterDate(start_date, end_date)
      .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
      .map(mask_s2_clouds)
      .median()
      .clip(region)
)

rgb_vis = {
    'min': 0.0,
    'max': 0.3,
    'bands': ['B4', 'B3', 'B2'],
}

map.add_ee_layer(rgb, rgb_vis, 'Sentinel-2 RGB')
display(map)

In [11]:
ndvi = rgb.normalizedDifference(['B8', 'B4']).rename('NDVI')

In [12]:
ndvi_vis = {
  'min': -1,
  'max': 1,
  'palette': ['blue', 'white', 'green']
}

map.add_ee_layer(ndvi, ndvi_vis, 'NDVI')
display(map)

In [13]:
ndwi = rgb.normalizedDifference(['B3', 'B8']).rename('NDWI')

ndwi_vis = {
  'min': -1,
  'max': 1,
  'palette': ['white', 'blue']
}

map.add_ee_layer(ndwi, ndwi_vis, 'NDWI')
display(map)

In [14]:
rain = (
    ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY')
      .filterDate(start_date, end_date)
      .select('precipitation')
      .sum()
      .clip(region)
)

rain_vis = {
    'min': 0,
    'max': 100,
    'palette': ['white', 'blue'],
}

map.add_ee_layer(rain, rain_vis, 'Precipitation')

display(map)

In [15]:
folium.LayerControl(collapsed = False).add_to(map)
display(map)